In [ ]:
import os

import azureml.core

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscriptoin-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="westus2")

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, 
                   resource_group = resource_group, 
                   workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config(path="../")
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

In [ ]:
# from azureml.core.authentication import InteractiveLoginAuthentication

# interactive_auth = InteractiveLoginAuthentication(tenant_id="my-tenant-id")

# from azureml.core import Workspace

# ws = Workspace(subscription_id="my-subscription-id",
#                resource_group="my-ml-rg",
#                workspace_name="my-ml-workspace",
#                auth=interactive_auth)

In [ ]:

 # Create the workspace using the specified parameters
 ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      sku = 'basic',
                      exist_ok = True)
 ws.get_details()

 write the details of the workspace to a configuration file to the notebook library
 ws.write_config()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "train-many-model"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=5,
                                                           max_nodes=20)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

---

## Next steps

Now that you've created your configuration file, you're all set to move on to the [01_Data_Preparation.ipynb](https://github.com/microsoft/solution-accelerator-many-models/blob/master/01_Data_Preparation/01_Data_Preparation.ipynb) notebook to prepare your datasets.